# Spam Filter using Naive Bayes
In this project we will make the computer learn to filter spam using the multinominal naive Bayes algorithm. We are going to use data from around 5000 text messages which have been classified as spam and ham(not spam). From there the computer will be able to classify single words for the probability of being used in spam messages. 

So much for a short intro, let's get to work!

In [1]:
#importing libraries
import numpy as np
import pandas as pd

#Reading in the text messages
texts = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['label','SMS'])
texts.head()

,label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [2]:
texts.describe()

,label,SMS
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [3]:
texts.label.value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: label, dtype: float64

# Ham and spam
As we can see there are two columns. One with a classification, 'ham' or 'spam' and the other with an actual message. Our dataset contains 5572 messages, where 86.5% is spam. The sample is not completely representative for the situation in 2020 anymore. Read for example [this article](https://www.ctia.org/news/protecting-consumers-by-stopping-text-messaging-spam#:~:text=Today%2C%20less%20than%20three%20percent,that%20consumers%20want%20to%20receive.) that found that only three percent of all messages is spam. 

Nevertheless for this training project it is a good file to start with. It could be that there are countries where text spam is a lot more common for example.

## Training the algorithm
In order for the machine to be trained properly we need to have a **training** dataset and a **test** dataset. Therefore we are going to shuffle the messages before taking two samples. 80% from the set will be for training and 20% for the test. 

## Goal
The goal for this project is to create an algorithm which predicts, the category ham or spam for a new message, with 80% accuracy. The test data will serve as 'new messages' to be classified by the algorithm. 

In [4]:
#Here the ways of the dataset are split, they are holding hands, but at some point the two samples will have to be able to live
#on their own. Being perfect for training and testing. First, the amounts:

print(5572*0.8) 
5572*0.20

4457.6


1114.4

In [5]:
#Our training set will contain 4458 rows and the test set 1114, let's shuffle first. 
texts = texts.sample(frac=1,random_state=1)

#now we will split the set at row 4458
training = texts[0:4457].reset_index(drop=True)
test = texts[4458:].reset_index(drop=True)

In [6]:
#It's smart to double-check if the percentages are still similar in both sets. 
training.label.value_counts(normalize=True)

ham     0.86538
spam    0.13462
Name: label, dtype: float64

In [7]:
test.label.value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: label, dtype: float64

The percentages are nearly the same as in the original data! So we can move on to building the algorithm.

As a start we need to clean our training set a bit first. We are only interested in the words, hence we will removes all punctuation marks first. Then we will set all words to lower case to keep it uniform. 

In [8]:
#I got a nasty warning, so I am stopping the warnings.
import warnings
warnings.filterwarnings('ignore')

#removing non word characters and setting the strings to lower case
training['SMS'] = training['SMS'].str.replace('\W',' ').str.lower()

In [9]:
training.head()

,label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


In [10]:
#In order to extract the total vocabulary we will make every word an item and count them
training['SMS'] = training['SMS'].str.split()

#now let's iterate over the list of lists using a nested loop.
vocabulary = []
for row in training['SMS']:
    for word in row:
        vocabulary.append(word)
        
#To remove duplicates we transform the list to a set and back.
vocabulary = set(vocabulary)
vocabulary = list(vocabulary)

vocabulary[:5]

['rofl', 'tlp', 'tomorw', 'monos', 'computer']

In [11]:
#Here we are creating a dictionary of the occurance of every word per SMS.
word_counts_per_sms = {unique_word: [0] * len(training['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,rofl,tlp,tomorw,monos,computer,heehee,colleagues,brownies,correctly,galileo,...,deal,cupboard,qbank,palm,rents,copied,tms,526,philosophy,refilled
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## A big table of words
As we can see there are a lot of zero's in this dataset. But that's allright, it is showing all the unique values in the data. In total there are 7783 words. 

When we manage to combine the original table with this data we can see whether a word is categorized as spam or not. 

In [12]:
#This should have been the last step of the cleaning phase. 
training_set = pd.concat([training,word_counts],axis=1)
print(training_set.shape)
training_set.head(40)

(4457, 7784)


,label,SMS,rofl,tlp,tomorw,monos,computer,heehee,colleagues,brownies,...,deal,cupboard,qbank,palm,rents,copied,tms,526,philosophy,refilled
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,ham,"[ok, i, thk, i, got, it, then, u, wan, me, 2, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,ham,"[i, want, kfc, its, tuesday, only, buy, 2, mea...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,ham,"[no, dear, i, was, sleeping, p]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,ham,"[ok, pa, nothing, problem]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,ham,"[ill, be, there, on, lt, gt, ok]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#After this we will have our definite cleaned set.
training_set = training_set[training_set['label'].notna()]
training_set.label.value_counts(dropna=False)

ham     3857
spam     600
Name: label, dtype: int64

## The algorithm

Let's try to make sense of the equations that are needed to build the filter. 

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam) \\
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}

This is the Naive Bayes algorithm, but there are several things added. The first thing to note is that it are actually two different formulae. The upper formula calculates the probability of a set of words (a text) being spam, and the second calculates the probability of ham. In the end the classification will be measured by comparing the resulting values. e.g. if the value for \\(P(Spam | w_1,w_2, ..., w_n)\\) (which means probability for spam given the set of words) is bigger than \\(P(Ham | w_1,w_2, ..., w_n) \\) the message will be classified as spam.

The \\(\propto\\)-sign means proportionate. This shows the difference between the Bayes theorem and the Naive Bayes algorithm.

\begin{equation}
P(B|A) = \dfrac{P(B) \cdot P(A|B)}{P(B) \cdot P(A|B) + 
 P(\overline{B}) \cdot P(A|\overline{B})}
\end{equation}

The lower sum, which is the law of total probability, is the same for both formulae. Hence we can say that the ratio still stays the same, but it does not equal the real probability anymore. 

The algorithm still needs to be corrected for words that have not been in any message before. So if the test comes up with new words (which will happen) the probabilities for a word are not able to be computed if it gives a 0 value. Therefore we are adding **'Laplace' smoothing** which means we will add \\(\propto=1\\) to every value above the fraction and divide by the total amount of words in the second fraction. The formulae will look like this. 

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} \\
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

If you did not get all this fancy math part, that's fine you're not missing a lot. It is only to show what I did for an interested and mathematically gifted audience. 

In [14]:
#Firstly we will calculate P(spam) and P(ham).
p_ham = len(training_set[training_set['label']=='ham'])/len(training_set)
p_spam = len(training_set[training_set['label']=='spam'])/len(training_set)
print('p_ham:',p_ham)
print('p_spam:', p_spam)

#Then N_spam, the amount of words that are spam. 
N_spam = 0
for words in training_set[training_set['label']=='spam']['SMS']:
    N_spam += len(words)
print('N_spam:',N_spam)

#N_ham
N_ham = 0
for words in training_set[training_set['label']=='ham']['SMS']:
    N_ham += len(words)
print('N_ham:',N_ham)

#N_vocabulary
N_vocab = len(vocabulary)
print('N_vocabulary:',N_vocab)

#Laplace smoothing
alpha = 1
print('alpha:',alpha)

p_ham: 0.8653803006506618
p_spam: 0.13461969934933812
N_spam: 15190
N_ham: 57233
N_vocabulary: 7782
alpha: 1


## Constant values
Above we have calculated the constant values of our algorithm. However the values for the words are constant as well. The training set is not going to change. So \\(P(\text{"viagra"}|Spam)\\) is not going to change either. Therefore it makes sense to calculate these values already. This saves computing power in the end. This means we should compute the values for every word with this formula:

\begin{equation}
P(\text{"viagra"}|Spam) = \frac{N_{"viagra"|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

The same calculation can be made for \\(P(\text{"viagra"}|Ham)\\). This means that every word will get two values in our training set. 

The fact that we calculate so many values before even beginning the classification of new messages makes the Naive Bayes algorithm very fast (especially compared to other algorithms). When a new message comes in, most of the needed computations are already done, which enables the algorithm to almost instantly classify the new message.

If we didn't calculate all these values beforehand, then all these calculations would need to be done every time a new message comes in. Imagine the algorithm will be used to classify 1,000,000 new messages. Why repeat all these calculations 1,000,000 times when we could just do them once at the beginning?

In [15]:
#Let's initiate two dictionaries for spam and ham.
spam_words = {unique_word:0 for unique_word in vocabulary}
ham_words = {unique_word:0 for unique_word in vocabulary}

#Now we need to separate the training_set into two sets. One for ham and one for spam.
spam = training_set[training_set['label']=='spam']
ham = training_set[training_set['label']=='ham']

#Time to build the formula.
for word in vocabulary:
    N_word_spam = spam[word].sum()
    spam_words[word] = (N_word_spam+alpha)/(N_spam+alpha*N_vocab)
    
    N_word_ham = ham[word].sum()
    ham_words[word] = (N_word_ham+alpha)/(N_ham+alpha*N_vocab)

In [16]:
spam_words #The formula has worked its magic, now we can move on to checking the new messages. 

{'rofl': 4.353125544140693e-05,
 'tlp': 0.0001305937663242208,
 'tomorw': 4.353125544140693e-05,
 'monos': 8.706251088281386e-05,
 'computer': 0.00021765627720703466,
 'heehee': 4.353125544140693e-05,
 'colleagues': 4.353125544140693e-05,
 'brownies': 4.353125544140693e-05,
 'correctly': 4.353125544140693e-05,
 'galileo': 4.353125544140693e-05,
 'exhausted': 4.353125544140693e-05,
 'reasonable': 4.353125544140693e-05,
 'ktv': 4.353125544140693e-05,
 'loooooool': 4.353125544140693e-05,
 'remembered': 4.353125544140693e-05,
 'shoot': 4.353125544140693e-05,
 'ask': 4.353125544140693e-05,
 'shagged': 0.0001305937663242208,
 'dramatic': 4.353125544140693e-05,
 'ill': 8.706251088281386e-05,
 'browse': 8.706251088281386e-05,
 'maths': 4.353125544140693e-05,
 'jolly': 4.353125544140693e-05,
 '09066364589': 0.0001305937663242208,
 'crammed': 4.353125544140693e-05,
 'dancin': 4.353125544140693e-05,
 'aretaking': 4.353125544140693e-05,
 'line': 0.001305937663242208,
 'speling': 4.353125544140693e

## Classification and word probability
The last step in the formula is to multiply the words and get \\(P(\text{"w"}|Spam)\\) and \\(P(\text{"w"}|Ham)\\)
Then after comparing the values to each other the algorithm should reach its conclusion.

In [17]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in spam_words:
            p_spam_given_message *= spam_words[word]
        else:
            pass
        if word in ham_words:
            p_ham_given_message *= ham_words[word]
        else:
            pass

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [18]:
#Now let's test our freshly born algorithm.
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3489598779101096e-25
P(Ham|message): 1.9380782419077522e-27
Label: Spam


In [19]:
#It seems they are not that keen on spamming 'viagra' per SMS. As you can see, the word isn't in any messages.
classify('viagra')

P(Spam|message): 0.13461969934933812
P(Ham|message): 0.8653803006506618
Label: Ham


In [20]:
classify('get hot babes now')

P(Spam|message): 2.4037438032626026e-13
P(Ham|message): 6.130179951788986e-14
Label: Spam


In [21]:
#Even a normal sentence with lots of spam words gets classified rightly.
classify('Hey could you come over hot babe?') 

P(Spam|message): 1.9110091775697878e-24
P(Ham|message): 3.9083559665848125e-21
Label: Ham


## Testing the classify function
The classify formula seems to work quite well! So it is time to tell its accuracy level on the test values. 
First we will apply our formula on the texts in the test file, save the result in a column and then compare the results. In order for the result to be right we need to return 'spam' or 'ham' instead of printing something like above.

In [22]:
import re

def classify_test(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in spam_words:
            p_spam_given_message *= spam_words[word]
        else:
            pass
        if word in ham_words:
            p_ham_given_message *= ham_words[word]
        else:
            pass

    if p_ham_given_message > p_spam_given_message:
        return 'ham'                                  #Here we can see how this formula is different, it returns.
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'undecided'

In [23]:
test['predicted'] = test['SMS'].apply(classify_test)
test.head()

,label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [24]:
#Now let's measure the accuracy
correct = 0
total = len(test)
for row in test.iterrows():
    row = row[1]
    if row['label'] == row['predicted']:
        correct += 1

accuracy = correct/total*100
print('correct:',correct)
print('total:',total)
print('The accuracy of the multinominal Naive Bayes spamfilter is {}%'.format(accuracy))

correct: 1100
total: 1114
The accuracy of the multinominal Naive Bayes spamfilter is 98.74326750448833%


In [25]:
#Only 14 misses! It would be interesting to examine those closely
mistakes = []
for row in test.iterrows():
    row = row[1]
    if row['label'] != row['predicted']:
        mistakes.append(row['SMS'])
        
print(mistakes)

['Not heard from U4 a while. Call me now am here all night with just my knickers on. Make me beg for it like U did last time 01223585236 XX Luv Nikiyu4.net', "More people are dogging in your area now. Call 09090204448 and join like minded guys. Why not arrange 1 yourself. There's 1 this evening. A£1.50 minAPN LS278BB", 'Unlimited texts. Limited minutes.', '26th OF JULY', 'Nokia phone is lovly..', 'A Boy loved a gal. He propsd bt she didnt mind. He gv lv lttrs, Bt her frnds threw thm. Again d boy decided 2 aproach d gal , dt time a truck was speeding towards d gal. Wn it was about 2 hit d girl,d boy ran like hell n saved her. She asked \'hw cn u run so fast?\' D boy replied "Boost is d secret of my energy" n instantly d girl shouted "our energy" n Thy lived happily 2gthr drinking boost evrydy Moral of d story:- I hv free msgs:D;): gud ni8', 'No calls..messages..missed calls', 'We have sent JD for Customer Service cum Accounts Executive to ur mail id, For details contact us', "Oh my god!

# Conclusion
It has to be said that these messages are actually quite ambiguous. I am having a hard time distinguishing some myself. Which means the algorithm which is basically quite simple is performing really well. I was quite surprised to see it score over 98% when all we had asked for was 80%. Thank you for taking the time to read my code, have a great day.